In [ ]:
!pip install transformers
!pip install keras
!pip install Sentencepiece
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install datasets
!pip install -U scikit-learn

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["WANDB_DISABLED"] = "true"

In [7]:
import transformers
import torch
#import keras
from datasets import load_dataset, concatenate_datasets

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functions import *

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

In [3]:
allgemein = pd.read_csv("new_data_allgemein.csv", sep=";", index_col="Unnamed: 0")
psycho = pd.read_csv("new_data_PsychoTherapie.csv", sep=";", index_col="Unnamed: 0")
new = pd.concat([allgemein, psycho])
new.head()

,index,title,thread_id,post_id,Satz-Id,date,username,split
0,0,Milli's Weg - über's MMK zur OP,98262,1962865,1,"17. Juli 2022, 23:34",Milli84,so..
1,1,Milli's Weg - über's MMK zur OP,98262,1962865,2,"17. Juli 2022, 23:34",Milli84,seit gut 3 Wochen habe ich nach jeder 2-3.
2,2,Milli's Weg - über's MMK zur OP,98262,1962865,3,"17. Juli 2022, 23:34",Milli84,Mahlzeit üblen Durchfall.
3,3,Milli's Weg - über's MMK zur OP,98262,1962865,4,"17. Juli 2022, 23:34",Milli84,Die Blähungen werden wieder schlimmer..
4,4,Milli's Weg - über's MMK zur OP,98262,1962865,5,"17. Juli 2022, 23:34",Milli84,"für morgen früh steht auf dem Plan, dass ich..."


In [4]:
new.to_csv("new_data_both.csv")

In [8]:
dataset = load_dataset("csv", data_files="new_data_both.csv", split="train")

Using custom data configuration default-40b45abc75636b27
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-40b45abc75636b27/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [4]:
sentences = new["split"].to_list()
#sentences_test = gpt_test["Sentence"].to_list()
splitted_sent = np.array_split(sentences, 10)

In [15]:
from transformers import TextClassificationPipeline
from transformers.pipelines.pt_utils import KeyDataset

cuda = torch.device('cuda')
model = AutoModelForSequenceClassification.from_pretrained("../MI_Data/Bert_Finetuning/models/FN_finetuned_gbert/model", local_files_only=True).cuda()
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

In [17]:
sents = []
label_0_scores = []
label_1_scores = []

'''for entry in splitted_sent:
    with pipe.device_placement():
        torch.cuda.empty_cache()
        predictions_train = pipe(list(entry), truncation=True, batch_size=4)
        for entry in predictions_train:
            sents.append(entry)
            label_0_scores.append(entry[0]['score'])
            label_1_scores.append(entry[1]['score'])'''

for out in pipe(KeyDataset(dataset, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])

In [18]:
#len(label_0_scores)
pred_train_df = pd.DataFrame({"split": dataset[:]['split'], "0":label_0_scores, "1": label_1_scores })

In [19]:
pred_train_df

,split,0,1
0,so..,0.998540,0.001460
1,seit gut 3 Wochen habe ich nach jeder 2-3.,0.030142,0.969858
2,Mahlzeit üblen Durchfall.,0.028735,0.971265
3,Die Blähungen werden wieder schlimmer..,0.972059,0.027941
4,"für morgen früh steht auf dem Plan, dass ich...",0.866028,0.133972
...,...,...,...
987,"Was ich damit sagen will, nicht aufgeben.",0.991707,0.008293
988,"Jeder Stein der sich in den Weg stellt, ist e...",0.964459,0.035541
989,"Wenn nicht, na dann stellt sich der gleiche S...",0.998407,0.001593
990,Ist nicht weiter tragisch ....,0.003337,0.996663


In [20]:
pred_train_df.to_csv("FN_mot_predictions.csv")

In [9]:
pred_train_df = pd.read_csv("FN_mot_predictions.csv", index_col="Unnamed: 0")

In [10]:
pred_train_df.describe()

,0,1
count,992.000000,992.000000
mean,0.746963,0.253037
std,0.398731,0.398731
min,0.001567,0.000593
25%,0.475100,0.001922
50%,0.993599,0.006401
75%,0.998078,0.524900
max,0.999407,0.998433


In [17]:
mot = pred_train_df[pred_train_df["1"] > 0.95]

In [18]:
len(mot)

166

In [19]:
mot.to_csv("mot_095.csv")

In [109]:
mot_sent = mot["split"].to_list()
splitted_mot = np.array_split(mot_sent, 10)

cuda = torch.device('cuda')
model = AutoModelForSequenceClassification.from_pretrained("../Ablation_Studies/classif/valence/bert-base-german-cased_model", local_files_only=True).cuda()
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

label_0_scores = []
label_1_scores = []

for entry in splitted_mot:
    with pipe.device_placement():
        torch.cuda.empty_cache()
        predictions_train = pipe(list(entry), truncation=True, batch_size=4)
        for entry in predictions_train:
            label_0_scores.append(entry[0]['score'])
            label_1_scores.append(entry[1]['score'])

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


In [110]:
pred_mot_df = pd.DataFrame({"split": mot_sent, "0":label_0_scores, "1": label_1_scores })

In [43]:
pred_mot_df.to_csv("mot_valence_preds.csv")

In [28]:
pred_mot_df = pd.read_csv("valence_predictions.csv")

In [29]:
pos = pred_mot_df[pred_mot_df["1"] > 0.95].copy()

In [30]:
neg = pred_mot_df[pred_mot_df["0"] > 0.95].copy()

In [31]:
pos["labels"] = 1
neg["labels"] = 0

In [32]:
mot = pd.concat([pos, neg]).sample(frac=1)
mot.head()

,Unnamed: 0,split,0,1,labels
765,765,"Ich hoffe, du fühlst dich dort wohl und es w...",0.000428,0.999572,1
71,71,Da liege ich aber gerade mit der nächsten WHO...,0.001104,0.998896,1
903,903,"Das sind ja durchaus auch Ansätze, die sich ...",0.000467,0.999533,1
197,197,Alles Gute jedenfalls für dich!,0.000893,0.999107,1
481,481,"Das Gewicht selbst ignorierte ich, da ich mic...",0.995543,0.004457,0


In [33]:
mot.drop(columns={"0","1"},inplace=True)
mot.rename(columns={"split": "Sentence"}, inplace=True)

In [34]:
mot.head()

,Unnamed: 0,Sentence,labels
765,765,"Ich hoffe, du fühlst dich dort wohl und es w...",1
71,71,Da liege ich aber gerade mit der nächsten WHO...,1
903,903,"Das sind ja durchaus auch Ansätze, die sich ...",1
197,197,Alles Gute jedenfalls für dich!,1
481,481,"Das Gewicht selbst ignorierte ich, da ich mic...",0


In [35]:
orig = pd.read_csv("../Ablation_Studies/forum/valence/train_valence_preproc.csv", index_col="Unnamed: 0")
orig.head()

,Sentence,labels,function_Sentence,nonfunction_Sentence,non_noun_adj,non_noun_verb_adj,non_verb,non_verb_aux,non_noun
14,Ich habe mich halt zusammengerissen und wenige...,1,Ich habe mich weniger der war sollte was,"halt zusammengerissen und gegessen , kurs teue...",Ich habe mich halt zusammengerissen und wenige...,"Ich habe mich halt und weniger , der war teuer...","Ich habe mich halt und weniger , der kurs war ...","Ich mich halt und weniger , der kurs teuer gen...",Ich habe mich halt zusammengerissen und wenige...
4164,ich habe so toll und erfolgreich 45kg abgenom...,1,ich habe,so toll und erfolgreich 45 kg abgenommen .,ich habe so toll und erfolgreich 45 abgenomm...,ich habe so toll und erfolgreich 45 .,ich habe so toll und erfolgreich 45 kg .,ich so toll und erfolgreich 45 kg .,ich habe so toll und erfolgreich 45 abgenomm...
3361,Schwimmen gehe ich nach wie vor und das war e...,1,ich das war es,Schwimmen gehe nach wie vor und auch schon .,gehe ich nach wie vor und das war es auch sc...,ich nach wie vor und das war es auch schon .,Schwimmen ich nach wie vor und das war es au...,Schwimmen ich nach wie vor und das es auch s...,gehe ich nach wie vor und das war es auch sc...
2741,Ich fühle mich als Dicke leider sauwohl.,0,Ich mich als,fühle Dicke leider sauwohl .,Ich fühle mich als leider sauwohl .,Ich mich als leider sauwohl .,Ich mich als Dicke leider sauwohl .,Ich mich als Dicke leider sauwohl .,Ich fühle mich als leider sauwohl .
696,"es war mir alles egal, denn ich war stolz, da...",1,es war mir alles ich war ich mit meinem,"egal , denn stolz , dass fetten Hinterteil i...","es war mir alles egal , denn ich war stolz ,...","es war mir alles egal , denn ich war stolz ,...","es war mir alles egal , denn ich war stolz ,...","es mir alles egal , denn ich stolz , dass ic...","es war mir alles egal , denn ich war stolz ,..."


In [36]:
orig = orig[["Sentence", "labels"]]

In [37]:
enhanced = pd.concat([orig, mot]).sample(frac=1)

In [38]:
enhanced.to_csv("enhanced_train_only_valence_pred_095.csv")

In [39]:
len(enhanced) - len(orig)

811

In [137]:
enhanced = pd.read_csv("enhanced_train_all_05.csv")
len(enhanced) - len(orig)

166